# hello sir/ma'am 
this is the start of the the approach of one of your  problem statement which was :
        Algorithmic Adaptation:
        These methods involve a variant of an algorithm that is designed to handle
        multi-label problems.
        Eg- Multi-Label KNNs, Multi-Label Decision Trees are good examples.

importing all the libraries 
(note some of them was added later in the below cell as i moved on)

In [11]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline



from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier




from sklearn.multioutput import ClassifierChain
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score,recall_score

# importing
the dataset , which was merged into one , to make my work easier

In [2]:
df=pd.read_csv(r"C:\RKT\CODE\python\projects\assignment\Data\final_movie_genre_v2.csv")
# Display first few rows
print("Original Data:")
display(df.head())

Original Data:


,title,overview,genre_ids,genres
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]","['Drama', 'Crime']"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","['Drama', 'Crime']"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","['Drama', 'Crime']"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","['Drama', 'History', 'War']"
4,12 Angry Men,The defense and the prosecution have rested an...,[18],['Drama']


In [3]:
df = df.dropna(subset=['overview'])
df['genres_list'] = df['genres'].apply(ast.literal_eval)


This was keeping the same as the tranformation , for multilevel binarizer


In [4]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genres_list'])

In [5]:
print(f"\nUnique Classes ({len(mlb.classes_)}): {mlb.classes_}")
print(f"Shape of Y: {y.shape}")


Unique Classes (18): ['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Drama' 'Family'
 'Fantasy' 'History' 'Horror' 'Music' 'Mystery' 'Romance'
 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']
Shape of Y: (9999, 18)


this was same as , other , but her i used ngram range becoasue i needed to look at words as knight and pairs of word like the dark knight 

# note


also the min for , if words apprar less than 2 times and more than 90 percent pof the time ignore it 
it is like setiing a threshold range...not too low and not to high similiar to removing the outliers in a numerical column by taking a percenitle 

In [6]:
tfidf = TfidfVectorizer( ngram_range=(1, 2),min_df=2,max_df=0.9, stop_words='english')
X = tfidf.fit_transform(df['title'])

In [51]:
import joblib 
joblib.dump(tfidf, 'tfidf_vectorizer_v2.pkl')

['tfidf_vectorizer_v2.pkl']

In [7]:
print(f"Shape of X: {X.shape}")

Shape of X: (9999, 3414)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Multi-Label k-NN Model

In [17]:
knn = OneVsRestClassifier(
    KNeighborsClassifier(
        n_neighbors=20,      # hyperparameter
        metric='cosine',
        weights='distance'
    )
)

knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)


In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n📌 {model_name}")
    print("Hamming Loss:", hamming_loss(y_true, y_pred))
    print("Exact Match Accuracy:", accuracy_score(y_true, y_pred))
    print("Micro F1:", f1_score(y_true, y_pred, average='micro'))
    print("Macro F1:", f1_score(y_true, y_pred, average='macro'))
    print("Micro Precision:", precision_score(y_true, y_pred, average='micro'))
    print("Micro Recall:", recall_score(y_true, y_pred, average='micro'))

evaluate_model(y_test, y_pred_knn, "Multi-Label KNN")


📌 Multi-Label KNN
Hamming Loss: 0.13577777777777778
Exact Match Accuracy: 0.0745
Micro F1: 0.32653623587765224
Macro F1: 0.2472532641641796
Micro Precision: 0.5984848484848485
Micro Recall: 0.22451686244789693


In [30]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("movie_predictor_V2")

with mlflow.start_run(run_name="Multi _label KNN"):

    acc = accuracy_score(y_test, y_pred_knn)

    mlflow.log_params({ "n_neighbors":20,  
        "metric":'cosine',
        "weights":'distance'})


    #   metrics
    mlflow.log_metrics({
    "hamming_loss": hamming_loss(y_test, y_pred_knn),
    "exact_match_accuracy": accuracy_score(y_test, y_pred_knn),
    "micro_f1": f1_score(y_test, y_pred_knn, average="micro"),
    "macro_f1": f1_score(y_test, y_pred_knn, average="macro"),
    "micro_precision": precision_score(y_test, y_pred_knn, average="micro"),
    "micro_recall": recall_score(y_test, y_pred_knn, average="micro")
})

  
    mlflow.sklearn.log_model(
        sk_model=knn,
        artifact_path="model_knn",
        registered_model_name="MultiLabelKNN"
    )

2026/02/10 18:17:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Successfully registered model 'MultiLabelKNN'.
2026/02/10 18:17:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MultiLabelKNN, version 1


🏃 View run Multi _label KNN at: http://127.0.0.1:5000/#/experiments/5/runs/83ec76336e4b47fc8b8dbd7bb77de16b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


Created version '1' of model 'MultiLabelKNN'.


as u can see , the prediction where goin good , but it is not as much asccurate 

In [20]:
def predict_genres(title, model):
    title_clean = title.lower()
    title_vec = tfidf.transform([title_clean])
    pred = model.predict(title_vec)
    return mlb.inverse_transform(pred)[0]

movie_title = "Rush Hour"

print("KNN Prediction:", predict_genres(movie_title, knn))

KNN Prediction: ('Action', 'Comedy', 'Crime')


needed to lower threshold for the classifier, cuz it didnt worked for thr dark knhight lol , i had to keep gettting it , untill i got to the thresshold i required  

In [24]:
def predict_genres(title, model):
    title_clean = title.lower()
    title_vec = tfidf.transform([title_clean])
    
    # Fixed internal threshold
    THRESHOLD = 0.4   # you can change this value here
    
    probs = model.predict_proba(title_vec)[0]
    
    labels = [
        mlb.classes_[i]
        for i, p in enumerate(probs)
        if p >= THRESHOLD
    ]
    
    return labels


movie_title = "The Dark Knight"

print("KNN Prediction:", predict_genres(movie_title, knn))


KNN Prediction: ['Action', 'Crime', 'Drama', 'Thriller']


In [53]:
import joblib

joblib.dump(knn, 'knn.pkl')

['knn.pkl']

# Multi-Label Decision Tree

Decision Trees don't assume a straight-line relationship; they split data based on if/then lgic, which handles complex word combinations well.
They handle high-dimensional data (like the thousands of columns created by your TfidfVectorizer) relatvely well without needing the massive computational power of a Neural Network.

In [ ]:
dt = OneVsRestClassifier(
    DecisionTreeClassifier(
        criterion='gini',
        max_depth=100,
        min_samples_split=15,
        min_samples_leaf=4,
        random_state=42
    )
)

dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)


In [47]:
evaluate_model(y_test, y_pred_dt, "Multi-Label Decision Tree")


📌 Multi-Label Decision Tree
Hamming Loss: 0.1421388888888889
Exact Match Accuracy: 0.035
Micro F1: 0.20034380371933114
Macro F1: 0.16950270062693787
Micro Precision: 0.5718108831400536
Micro Recall: 0.12144751799924214


In [49]:
def predict_genres_dt(title, model):
    title_clean = title.lower()
    title_vec = tfidf.transform([title_clean])

    THRESHOLD = 0.20  

    probs_per_label = model.predict_proba(title_vec)

    labels = []
    for i, probs in enumerate(probs_per_label):
        if probs[0][1] >= THRESHOLD:
            labels.append(mlb.classes_[i])

    # Safety net: ensure at least one genre
    if not labels:
        max_idx = np.argmax([p[0][1] for p in probs_per_label])
        labels = [mlb.classes_[max_idx]]

    return labels


movie_title = "Rush Hour"

print("DT Prediction:", predict_genres(movie_title,dt))

DT Prediction: ['Comedy', 'Crime', 'Drama']


In [52]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("movie_predictor_V2")

with mlflow.start_run(run_name="Multi _label DT"):

    acc = accuracy_score(y_test, y_pred_dt)

    mlflow.log_params({"criterion":"gini",
        "max_depth":100,        # controls overfitting
        "min_samples_split":15,
        "min_samples_leaf":4,
        "random_state":42})


    #   metrics
    mlflow.log_metrics({
    "hamming_loss": hamming_loss(y_test, y_pred_dt),
    "exact_match_accuracy": accuracy_score(y_test, y_pred_dt),
    "micro_f1": f1_score(y_test, y_pred_dt, average="micro"),
    "macro_f1": f1_score(y_test, y_pred_dt, average="macro"),
    "micro_precision": precision_score(y_test, y_pred_dt, average="micro"),
    "micro_recall": recall_score(y_test, y_pred_dt, average="micro")
})

  
    mlflow.sklearn.log_model(
        sk_model=dt,
        artifact_path="model_dt",
        registered_model_name="MultiLabeldt"
    )

2026/02/10 21:05:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Successfully registered model 'MultiLabeldt'.
2026/02/10 21:05:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MultiLabeldt, version 1


🏃 View run Multi _label DT at: http://127.0.0.1:5000/#/experiments/809687645743597975/runs/af81a534a00c42bfa54f9913f65623d0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809687645743597975


Created version '1' of model 'MultiLabeldt'.


as u can see that modell knn is better we will be having that , got to this decision after looking at merics in ml flow at the same time 
